<img src="../rsag_convex.png" alt="algoconvex" />
<img src="../x_update.png" alt="x_update" />
<img src="../mean.png" alt="mean" />
<img src="../rsag_composite.png" alt="algo" />

__Parameters :__
- $\alpha$: (1-$\alpha$) weight of aggregated x on current state, i.e. momentum
- $\lambda$: learning rate
- $\beta$: change for aggregated x
- $p_k$ termination probability



In [13]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [12]:
import sys
sys.path.append('../')

# Model Implementation

### Metrics

In [162]:
def evaluate_acc(truth, pred):
  truth = [truth]
  dif = np.abs([1-pred[i]+truth[i] for i in range(len(pred))])
  return np.sum(1-dif) / float(len(pred))


### Line

In [181]:
class Line():
    """
        Linear Model with two weights w0 (intercept) and w1 (slope)
    """
    def __init__(self, num_classes=10, rsag=False):
        self.params = None
        self.aggr_params = None
        self.rsag = rsag
        
    def predict(self,x):
        return self.params[0] + self.params[1]*x

    def fit(self, x, y, optimizer):
        N = len(x)

        def gradient(x, y, params):

            total = [0,0]
            for x_temp,y_temp in zip(x,y):

                w = params[1]
                b = params[0]

                yh = b + w*x_temp

                db = (yh - y_temp)
                dw = x_temp*(yh - y_temp)
                train_acc = evaluate_acc(y_temp, yh)
                # print(train_acc)
                
                total = [total[0] + db, total[1] + dw]

            gradient = [t/N for t in total]
            return gradient, train_acc

        if self.params is None:
            self.params = [np.random.uniform(0,1,1) for _ in range(2)]

            if self.rsag:
                self.aggr_params = [np.copy(w) for w in self.params]

        if self.rsag:
            self.params, self.aggr_params, train_acc = optimizer.mini_batch_step(gradient, x, y, self.params, self.aggr_params)
        else:
            self.params, train_acc = optimizer.mini_batch_step(gradient, x, y, self.params)

        return train_acc
    
    def __str__(self):
        return f"y = {self.params[0]} + {self.params[1]}*x"





# RSAG

In [182]:
lr_lamda = lambda lr, t: lr/(1+t)

In [183]:
class RSAG:

    def __init__(self, 
                 learning_rate=.001, 
                 alpha=0.009, 
                 beta=.000009, 
                 max_iters=200, 
                 epsilon=1e-8, 
                 lr_fn = None,
                alpha_fn = None,
                beta_fn = None,
                 ):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.alpha = alpha  # momentum param
        self.beta = beta 

        self.lr_fn = lr_fn
        self.alpha_fn = alpha_fn
        self.beta_fn = beta_fn

        self.update_params = False

        self.t = 0

    
    def set_update_params(self, update_params):
        self.update_params = update_params

    def mini_batch_step(self, 
                       gradient_fn,
                       x, 
                       y,
                       params, 
                       agg_params,
                       ):
        


        # if self.update_params:
        #     print('Update params')
        #     if self.lr_fn is not None:
        #         self.learning_rate = self.lr_fn(self.learning_rate, self.t)
        #         # print('New learning rate:', self.learning_rate)
        #     if self.alpha_fn is not None:
        #         self.alpha = self.alpha_fn(self.alpha, self.t)
        #         # print('New alpha:', self.alpha)
        #     if self.beta_fn is not None:
        #         self.beta = self.beta_fn(self.beta, self.t)
        #         # print('New beta:', self.beta)
        #     self.update_params = False

        grad = None


        proj_params = [(1-self.alpha) * a_p + self.alpha * p for p, a_p in zip(params, agg_params)]

        grad, temp_acc = gradient_fn(x, y, proj_params)

        train_acc = ( self.t, temp_acc ) 

        for p in range(len(params)):
            agg_params[p] -= self.beta * (grad[p])[0]
            params[p] -= self.learning_rate * (grad[p])[0]

            
        self.t += 1
            
        return params, agg_params, train_acc

# Convex

In [ ]:
from sklearn.neural_network import MLPClassifier
c_clf = MLPClassifier(hidden_layer_sizes=(), activation='identity', 
                    solver='sgd', 
                    alpha=0.0001, momentum=0.8, nesterovs_momentum=True,
                    batch_size=64, 
                    learning_rate='constant', learning_rate_init=0.09, 
                    verbose=True)
%time c_clf.fit(x_train, y_train)

In [191]:
class GD:
    """
        gd: will estimate the parameters w1 and w2 (here it uses least square cost function)
        model: the model we are trying to optimize using gradient descent
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        max_num_iteration: the number of iteration before we stop updating
    """    
    def __init__(self, learning_rate=.001):
        self.learning_rate = learning_rate
        self.t = 0

    def mini_batch_step(self, 
                        gradient_fn,
                        x, 
                        y,
                        params
                        ):
        
        # Updating the model parameters
        grad, temp_acc = gradient_fn(x, y, params)
        # chunk.append(temp_acc)
        train_acc =  ( self.t, temp_acc )

        for p in range(len(params)):
            params[p] -= self.learning_rate * (grad[p])[0]
        self.t += 1
        return params, train_acc

In [194]:
x_train = [1,2,3,4,5,6,7]
y_train = [1,2,3,4,5,6,7]

model = Line(num_classes=10, rsag=False)
optimizer = GD(learning_rate=.02)

for i in range(2000):
    model.fit(x_train, y_train, optimizer)
print(model.predict(x_train))
print(model)

[1.00021853 2.00016323 3.00010794 4.00005264 4.99999734 5.99994205
 6.99988675]
y = [0.00027383] + [0.9999447]*x


In [195]:
x_train = [1,2,3,4,5,6,7]
y_train = [1,2,3,4,5,6,7]

model = Line(num_classes=10, rsag=True)
optimizer = RSAG(learning_rate=.02, alpha=0.9, beta=0.001)

for i in range(2000):
    model.fit(x_train, y_train, optimizer)
print(model.predict(x_train))
print(model)

[0.97197172 2.01008872 3.04820571 4.08632271 5.1244397  6.16255669
 7.20067369]
y = [-0.06614527] + [1.03811699]*x
